In [121]:
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
from numba import jit
import pstats
import heapq
import time
import cProfile
import pandas as pd

In [122]:
puzzle_info_path = 'puzzle_info.csv'
puzzles_path = 'puzzles.csv'
sample_submission_path = 'sample_submission.csv' # change name across boardsample_submission_path
puzzle_info_df = pd.read_csv(puzzle_info_path)
puzzles_df = pd.read_csv(puzzles_path)
sample_submission_df = pd.read_csv(sample_submission_path)

In [123]:
# Parsing the initial_state and solution_state columns
# Converting the semicolon-separated string values into lists of colors
puzzles_df['parsed_initial_state'] = puzzles_df['initial_state'].apply(lambda x: x.split(';'))
seen = {}

for i in range(len(puzzles_df['parsed_initial_state'])):
    for j in range(len(puzzles_df['parsed_initial_state'][i])):
        if puzzles_df['parsed_initial_state'][i][j] not in seen:
            seen[puzzles_df['parsed_initial_state'][i][j]] = len(seen)
        puzzles_df['parsed_initial_state'][i][j] = seen[puzzles_df['parsed_initial_state'][i][j]]

puzzles_df['parsed_solution_state'] = puzzles_df['solution_state'].apply(lambda x: x.split(';'))

for i in range(len(puzzles_df['parsed_solution_state'])):
    for j in range(len(puzzles_df['parsed_solution_state'][i])):
        puzzles_df['parsed_solution_state'][i][j] = seen[puzzles_df['parsed_solution_state'][i][j]]

In [124]:
import json

# Converting the string representation of allowed_moves to dictionary
puzzle_info_df['allowed_moves'] = puzzle_info_df['allowed_moves'].apply(lambda x: json.loads(x.replace("'", '"')))

# Selecting an example puzzle type and displaying its allowed moves
example_puzzle_type = puzzle_info_df['puzzle_type'].iloc[0]
example_allowed_moves = puzzle_info_df[puzzle_info_df['puzzle_type'] == example_puzzle_type]['allowed_moves'].iloc[0]

In [125]:
def getInversePerm(arr):
    # gets the inverse move for a certain move
    res = [0 for i in range(len(arr))]
    for i in range(len(arr)):
        res[arr[i]] = i
    return res
    
# type : (np.array(move_perm_i), np.array(name_i))
puz_info = {}

# type : {move : perm}
move_to_perm = {}

for i in range(len(puzzle_info_df)):
    puz_info[puzzle_info_df['puzzle_type'][i]] = [[], []]
    move_to_perm[puzzle_info_df['puzzle_type'][i]] = {}
    
    for j in puzzle_info_df['allowed_moves'][i].keys():
        puz_info[puzzle_info_df['puzzle_type'][i]][1].append(j)
        puz_info[puzzle_info_df['puzzle_type'][i]][0].append(np.array(puzzle_info_df['allowed_moves'][i][j]))

        # puz_info[puzzle_info_df['puzzle_type'][i]][1].append(str('-' + j)) # might be the opposite
        # puz_info[puzzle_info_df['puzzle_type'][i]][0].append(np.array(getInversePerm(puzzle_info_df['allowed_moves'][i][j])))

        # move_to_perm[puzzle_info_df['puzzle_type'][i]][str('-' + j)] = np.array(getInversePerm(puzzle_info_df['allowed_moves'][i][j]))
        move_to_perm[puzzle_info_df['puzzle_type'][i]][j] = np.array(puzzle_info_df['allowed_moves'][i][j])

# move_to_perm['cube_2/2/2']

In [126]:
@jit(nopython=True, parallel = True, fastmath = True)
def hash_perm(perm):
    base = 9973
    modb = 1000000007
    modc = 1000000009

    B, C = 0, 0
    for i in perm:
        B = (B * base) % modb + i
        C = (C * base) % modc + i

    return (B, C)

def dist(a, b):
    return np.count_nonzero(a != b)

In [127]:
# 1e6 ~ 2 seconds
mx_mem = int(1e7)
mem_idx = 0

last_state = np.zeros(mx_mem, dtype=int)
last_move = np.zeros(mx_mem, dtype=int)

print(last_state)

[0 0 0 ... 0 0 0]


In [128]:
def chunk_search(idx, initial_state = None, chunk_time = 1, next_states = set()):
    global mem_idx

    if initial_state is None:
        initial_state = np.array(puzzles_df['parsed_initial_state'][idx])
    
    goal_state = np.array(puzzles_df['parsed_solution_state'][idx])
    max_dist = puzzles_df['num_wildcards'][idx]
    move_perm = np.array(puz_info[puzzles_df['puzzle_type'][idx]][0])

    n = len(initial_state)
    m = len(move_perm)

    avg_dist = np.array([n + 1 for i in range(500 * n)], dtype = np.float32)
    n_dist = np.array([0 for i in range(500 * n)])

    def upd_prune(nxt_p, nxt_dist, avg_dist, n_dist):
        if n_dist[nxt_p] == 0:
            avg_dist[nxt_p] = nxt_dist
            n_dist[nxt_p] = 1
            return True

        if n_dist[nxt_p] > 100000: # pruning here
            if nxt_dist > avg_dist[nxt_p]:
                return False
        
        avg_dist[nxt_p] *= n_dist[nxt_p] / (n_dist[nxt_p] + 1)
        avg_dist[nxt_p] += nxt_dist / (n_dist[nxt_p] + 1)
        n_dist[nxt_p] += 1
        return True
    
    pq = []
    heapq.heappush(pq, (0, mem_idx, initial_state, 0))  # (priority, mem_idx, state, path_length)
    mem_idx += 1

    best_branch = (dist(initial_state, goal_state), 0, initial_state, 0) # (score, mem_idx, state, path_length)

    considered = set()
    considered.add(hash_perm(initial_state))

    start_time = time.time()
    
    while mem_idx + m < mx_mem:
        if time.time() - start_time > chunk_time:
            break
        
        cur_p_plus_dist, cur_idx, cur_state, cur_p = heapq.heappop(pq) 

        for i in range(m):
            nxt_state = cur_state[move_perm[i]]
            nxt_hash = hash_perm(nxt_state)
            nxt_p = 1 + cur_p

            nxt_dist = dist(nxt_state, goal_state)
            
            # ok = upd_prune(nxt_p, nxt_dist,avg_dist,n_dist)
            # if not ok:
            #     continue

            if nxt_hash in considered:
                continue
            considered.add(nxt_hash)

            if nxt_hash in next_states:
                print("OK")

            if (nxt_dist == best_branch[0] and nxt_p < best_branch[3]) or nxt_dist < best_branch[0]:
                # print(nxt_dist, nxt_p)
                best_branch = (nxt_dist, mem_idx, nxt_state, nxt_p)
            
            last_state[mem_idx] = cur_idx
            last_move[mem_idx] = i

            if nxt_dist <= max_dist:
                # print("Number of pruned branches", pruned_branch)
                # print("Chunk length", cur_p)
                # print("Chunk Dist", nxt_dist)
                
                return best_branch
            
            priority = nxt_p + nxt_dist
            heapq.heappush(pq, (priority, mem_idx, nxt_state, 1+cur_p))
            mem_idx += 1
    return best_branch

mem_idx = 0
print(chunk_search(30))
# cProfile.run('chunk_search(30)')

/opt/homebrew/lib/python3.11/site-packages/numba/core/typed_passes.py:334: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see https://numba.readthedocs.io/en/stable/user/parallel.html#diagnostics for help.

File "../../../../../var/folders/kd/99328vjn0zlc1kls78_rwvdh0000gn/T/ipykernel_19449/3022860735.py", line 1:
<source missing, REPL/exec in use?>

  warnings.warn(errors.NumbaPerformanceWarning(msg,


(13, 233004, array([2, 2, 3, 2, 2, 2, 2, 0, 2, 3, 5, 4, 3, 3, 3, 3, 3, 2, 0, 3, 5, 4,
       4, 4, 4, 4, 4, 4, 0, 0, 5, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 5, 5, 3, 5, 5, 2, 5, 4, 5]), 20)


In [131]:
%%time
from collections import deque

def BFS(idx, initial_state = None):
    if initial_state is None:
        initial_state = np.array(puzzles_df['parsed_initial_state'][idx])
        initial_state = np.array([i for i in range(len(initial_state))])
    
    goal_state = np.array(puzzles_df['parsed_solution_state'][idx])
    max_dist = puzzles_df['num_wildcards'][idx]
    move_perm = np.array(puz_info[puzzles_df['puzzle_type'][idx]][0])

    n = len(initial_state)
    m = len(move_perm)
    
    Q = deque([(initial_state, None, [])])

    last_considered = {}
    
    while len(last_considered) < int(1e7):
        nQ = deque([])
        cur_considered = {}
        
        while len(Q) != 0:
            cur_state, last_move, path = Q.popleft()

            for i in range(m):
                if last_move == i:
                    continue
                    
                nxt_state = cur_state[move_perm[i]]
                nxt_hash = hash_perm(nxt_state)
                nxt_path = path.copy()
                nxt_path.append(i)
                
                if nxt_hash in last_considered:
                    # print(last_considered[nxt_hash], nxt_path)
                    continue

                if nxt_hash in cur_considered:
                    continue
                    
                cur_considered[nxt_hash] = nxt_path
    
                nQ.append((nxt_state, i, nxt_path))
        
        last_considered = last_considered | cur_considered
        
        Q = nQ
        # print(len(Q))
    return considered

print(len(BFS(30)))
# cProfile.run('chunk_search(30)')

NameError: name 'considered' is not defined

In [84]:
# optimize reference solution
reference_solution = pd.read_csv('submission.csv')
reference_solution

,id,moves
0,0,r1.-f1
1,1,f0.r1.f1.-d0.-d0.-f0.-r0.f0.d0
2,2,-d1.-r0.f0.-r1.f1.d1.-r1.-f0.d1.f0.d1.d1
3,3,-f0.d0.-r0.f0.-d0.-r0.d0.-f0.-r0.-f0
4,4,-r1.-f0.d0.r0.-d1.-d1.r1.d1.f0.r1.-d1.-r1
...,...,...
393,393,f19.f21.-f39.f20.f2.-f5.f7.-r3.f55.-f12.f65.-f...
394,394,-f31.-f22.f16.-f17.-f13.-f24.-f14.f2.f21.f44.f...
395,395,-r0.-f42.-f8.f16.-f49.f14.-f1.f56.f26.f35.f62....
396,396,f25.-f29.f46.f49.-f8.f27.f26.-f20.f2.-f20.f6.f...


In [59]:
temp = reference_solution['moves'][282]
print(temp[0:2000])

f7.-d1.r3.r25.-r2.r12.-f1.-r4.f25.r28.d7.f3.d8.-d6.-r19.-r1.r25.r32.-r20.-r16.-r23.-r32.d19.f24.-r20.f21.r25.f0.-f13.r16.d18.-r28.f9.-r8.r22.-r16.-f17.f22.-d27.f4.d25.r23.f29.r26.-d25.f27.-r4.f31.r13.-f10.-d25.-f4.-f31.f9.-f23.d28.-f5.-r24.f15.-f3.-r9.f32.-r18.-r0.f30.r18.-f32.-d24.-r12.r11.r22.-f16.-r9.-d17.r6.-r10.f31.f4.d25.r29.-r6.-d5.r18.-f23.d28.-f5.-r24.f15.-f3.f10.-r13.f3.f13.d6.r22.-f23.d28.-f5.r2.-f26.-d7.-r3.-r11.r12.d24.f32.-r18.-r29.-f30.f11.-d16.-r20.-d4.r5.-r22.-d6.-f13.r8.r15.f4.-r26.-r10.f6.-r24.r5.f28.-f9.-f12.-r6.d17.-f0.d32.-r7.f20.-d30.r12.d24.f32.-r18.-d16.-r20.d20.-f9.r28.-f7.-d14.-d32.r29.-f14.f1.-d28.-f2.-r18.d29.-r3.r22.r9.-r23.-r32.r2.-f26.-f25.d21.f16.f26.-r26.-r15.-r8.f13.d6.r22.-r8.r22.-r16.-f17.f22.-d27.r18.f29.d26.r5.-r3.-f2.-f11.-f23.-f22.-d25.-f4.-f31.-r25.-d0.-r23.-r32.r16.f11.d8.-f2.-f11.-f23.-f22.-d25.-f4.-f31.-r25.d3.-f7.r20.-d32.f0.-d17.r6.-f3.-d14.-d32.r29.-f14.r32.-r27.r17.r5.-f8.-f1.f14.-r29.d32.d14.f3.-r13.d1.-r6.d17.-f0.d32.-r20.f7.-d3.r25.f3

In [61]:
%%time
idx = 281

reference_nodes = []
hashed_nodes = set()
order_of_hashes = []
moves = move_to_perm[puzzles_df['puzzle_type'][idx]]
cur_moves = reference_solution['moves'][idx].split('.')

cur = np.array(puzzles_df['parsed_initial_state'][idx])

for j in cur_moves:
    cur_perm = moves[j]
    cur = cur[cur_perm]
    reference_nodes.append(cur)
    cur_hash = hash_perm(cur)
    # if cur_hash in hashed_nodes:
    #     print("yes")
    order_of_hashes.append(cur_hash)
    hashed_nodes.add(cur_hash)

CPU times: user 4.99 s, sys: 782 ms, total: 5.77 s
Wall time: 5.98 s


In [93]:
def isInverse(a, b):
    if b == '-' + a or a == '-' + b:
        return True
    return False

def removeInversePair(moves):
    res = []
    for i in moves:
        if len(res) != 0 and isInverse(res[len(res)-1], i):
            print('found')
            res.pop(len(res)-1)
        else:
            res.append(i)
    return res

a = removeInversePair(cur_moves)

In [62]:
print(order_of_hashes[0:10])

[(470129739, 292094178), (69400840, 502515716), (816957508, 927108116), (619995782, 207736656), (44151798, 147240293), (783438726, 820798202), (590688874, 559643326), (907990290, 459831859), (914661930, 250853257), (576361255, 551714091)]


In [65]:
all_hashes = set(hashed_nodes)
# print(all_hashes)
res = BFS(idx, chunk_time = 30, next_states = all_hashes)
print(len(res))

(470129739, 292094178)
OK
(69400840, 502515716)
OK
488535


In [98]:
def dig_through_memory(idx):
    res = []
    while idx != 0:
        res.append(last_move[idx])
        idx = last_state[idx]

    res = list(reversed(res))
    return res

def solve(idx, allotted_time = 10):
    global mem_idx
    
    start_time = time.time()
    move_name = np.array(puz_info[puzzles_df['puzzle_type'][idx]][1])
    max_dist = puzzles_df['num_wildcards'][idx]

    found = 0
    res = []
    cur = np.array(puzzles_df['parsed_initial_state'][idx])
    # chunk_time = 1
    chunk_time = allotted_time + 1
    
    while (time.time() - start_time < allotted_time and not found):
        # print("Start chunk")
        mem_idx = 0
        nxt_cur = chunk_search(idx, cur, chunk_time = chunk_time) 
        # larger chunk_time -> better solution, longer time
        # more breath on search

        if nxt_cur[0] <= max_dist:
            found = 1

        cur_moves = dig_through_memory(nxt_cur[1])
        res = res + cur_moves
        
        cur = nxt_cur[2]
        chunk_time *= 2
        # print("End chunk")
    
    if not found:
        return None
        
    for i in range(len(res)):
        res[i] = move_name[res[i]]
    return res

A = solve(5)
A

18 1
17 1
16 1
15 2
13 2
12 3
10 4
9 6
6 8
4 11
0 76


['r0',
 'f0',
 'd1',
 '-f0',
 'r0',
 '-f0',
 '-r0',
 'f0',
 'f0',
 '-r0',
 '-f0',
 'r0',
 'd1',
 'r0',
 '-d1',
 '-f0',
 '-d1',
 'f0',
 'd1',
 'r0',
 'd1',
 '-r0',
 '-d1',
 'r0',
 'd1',
 '-r0',
 '-d1',
 'r0',
 '-d1',
 '-r0',
 '-f0',
 'r1',
 'f0',
 '-r1',
 '-d0',
 '-r1',
 'd0',
 'r1',
 'f0',
 '-r1',
 '-f0',
 'r1',
 '-d0',
 '-r1',
 'f0',
 'd0',
 '-r0',
 '-d1',
 'f0',
 'd1',
 '-f0',
 '-r1',
 'd0',
 'r1',
 '-d0',
 'r1',
 '-f0',
 'd0',
 'f0',
 '-r1',
 'f0',
 'r1',
 '-f0',
 '-f0',
 'r0',
 'f0',
 '-r0',
 '-r0',
 'f0',
 'r0',
 'd1',
 'r0',
 '-d1',
 'f0',
 '-r0',
 '-f0']

# Step 6: Build submission format function

In [99]:
def format_solution_for_submission(puzzle_id, solution_moves):
    """
    Format the solution to a puzzle for submission.

    :param puzzle_id: The unique identifier of the puzzle.
    :param solution_moves: List of tuples representing the solution moves.
    :return: Formatted string suitable for submission.
    """
    formatted_moves = []
    for move in solution_moves:
        formatted_moves.append(move)

    # Joining the moves into a single string separated by periods
    return {'id': puzzle_id, 'moves': '.'.join(formatted_moves)}


# Step 7: Define solve function

In [100]:
last_submission_df = pd.read_csv('submission.csv')
last_submission_df

,id,moves
0,0,r1.-f1
1,1,f0.r1.f1.-d0.-d0.-f0.-r0.f0.d0
2,2,-d1.-r0.f0.-r1.f1.d1.-r1.-f0.d1.f0.d1.d1
3,3,-f0.d0.-r0.f0.-d0.-r0.d0.-f0.-r0.-f0
4,4,-r1.-f0.d0.r0.-d1.-d1.r1.d1.f0.r1.-d1.-r1
...,...,...
393,393,f19.f21.-f39.f20.f2.-f5.f7.-r3.f55.-f12.f65.-f...
394,394,-f31.-f22.f16.-f17.-f13.-f24.-f14.f2.f21.f44.f...
395,395,-r0.-f42.-f8.f16.-f49.f14.-f1.f56.f26.f35.f62....
396,396,f25.-f29.f46.f49.-f8.f27.f26.-f20.f2.-f20.f6.f...


In [101]:
from tqdm import tqdm

all_ok = ['cube_3/3/3', 'wreath_21/21', 'wreath_33/33']
res_df = []

for i in tqdm(range(len(puzzles_df))):
    res = None

    type = puzzles_df['puzzle_type'][i]
    if type in all_ok:
        res = solve(i, 90)
    
    if res is None:
        res = format_solution_for_submission(i, last_submission_df['moves'][i].split('.'))
    else:
        res = format_solution_for_submission(i, res)
    
    res_df.append(res)

  0%|                                                   | 0/398 [00:00<?, ?it/s]

42 1
38 2
36 3
35 4
33 5
32 6
29 6
28 9
27 11
26 12
25 10
24 13
23 14
22 14
21 13
20 15
20 14
20 11
19 16
16 18
15 18
13 20
13 17
12 23
11 25
11 22
10 27
8 30
6 46
5 612
3 672


  8%|███▎                                      | 31/398 [01:00<11:57,  1.96s/it]

0 1388
37 1
36 1
32 2
29 3
28 4
27 5
26 8
25 9
25 8
24 8
22 9
21 10
19 11
18 15
17 15
16 18
16 17
16 15
15 19
14 20
12 20
11 27
10 29
9 35


  8%|███▍                                      | 32/398 [01:03<12:05,  1.98s/it]

...

0 1103
37 1
35 2
34 3
32 4
31 3
29 4
28 8
27 9
27 7
26 8
25 10
24 10
22 14
21 15
20 9
19 11
18 14
17 17
17 16
15 17
14 20
14 15
13 18
9 24
6 26


 20%|███████▋                              | 80/398 [1:02:02<6:41:53, 75.83s/it]

34 1
32 1
28 2
26 3
25 4
24 7
23 8
21 9
20 10
19 13
18 13
17 13
16 15
15 15
15 14
12 17
9 20
6 91


 20%|███████▋                              | 80/398 [1:02:27<4:08:16, 46.84s/it]


KeyboardInterrupt: 

In [122]:
for i in tqdm(range(len(res_df), len(puzzles_df))):
    res = None

    type = puzzles_df['puzzle_type'][i]
    if type in all_ok:
        res = solve(i, 90)
    
    if res is None:
        res = format_solution_for_submission(i, last_submission_df['moves'][i].split('.'))
    else:
        res = format_solution_for_submission(i, res)
    
    res_df.append(res)


  0%|                                                   | 0/318 [00:00<?, ?it/s]

34 1
32 1
28 2
26 3
25 4
24 7
23 8
21 9
20 10
19 13
18 13
17 13
16 15
15 15
15 14
12 17
9 20
6 91



  0%|▏                                        | 1/318 [01:31<8:02:13, 91.27s/it]

39 1
37 1
36 1
34 2
33 2
31 3
29 4
25 7
24 8
23 7
22 10
21 14
19 14
19 13
18 12
17 14
16 15
15 19
14 20
14 14
13 21
10 20
9 26
6 88
5 1275
4 1279
3 1285



  1%|▎                                        | 2/318 [03:02<8:00:38, 91.26s/it]

...

20 1
19 2
18 3
17 4
16 5
15 6
14 7
13 9
12 8
10 11
8 16
6 18
4 25
2 80



 79%|██████████████████████████████▋        | 250/318 [1:38:51<01:19,  1.18s/it]

0 350
26 2
25 2
23 4
22 7
22 6
21 7
20 8
19 11
18 12
16 13
15 14
13 15
11 18
10 21
9 22
8 23
7 25
6 26
6 25
5 30
4 45
3 117
2 124
0 387
22 1
21 2
20 3
19 4
18 4
17 5
15 8
14 9
13 10
13 9
12 12
11 19
10 19
9 24
7 83
5 95
4 123
3 153
2 162
0 381
24 1
22 2
21 3
19 4
19 3
17 5
16 8
15 10
14 12
13 14
12 16
12 15
11 17
10 18
9 23
8 24
6 69
5 228
4 231
2 263
0 401
21 1
19 4
18 6
17 7
15 8
14 9
11 10
10 16
9 16
9 14
9 13
8 45
7 45
6 52
5 54
3 100
2 110
0 193
35 1
34 2
33 5
32 6
31 7
30 8
29 10
28 11
27 12
26 13
25 14
24 15
23 16
22 18
22 16
21 19
20 20
19 22
18 24
17 25
16 139
15 139
14 148
13 151
12 280
12 279
11 282
11 281
10 284
9 284
8 289
7 294
5 505
4 520
3 548
2 555
0 731
35 1
33 2
32 3
31 4
30 7
29 8
28 9
27 10
26 12
25 13
24 16
23 18
22 22
21 27
20 27
19 30
18 31
17 36
16 37
14 39
12 44
11 1202
9 1215
8 1328
7 1329
6 1342
5 1385
4 1388
3 1432
2 1435
0 1449
31 1
30 2
29 3
28 6
27 8
26 10
26 9
25 10
24 12
22 13
21 15
20 17
20 16
19 18
19 17
18 18
17 20
16 20
15 20
14 24
14 23
13 26
12 3


100%|███████████████████████████████████████| 318/318 [1:39:13<00:00, 18.72s/it]

8 906


In [125]:
res_df = pd.DataFrame(res_df)
res_df

,id,moves
0,0,r1.-f1
1,1,f0.r1.f1.-d0.-d0.-f0.-r0.f0.d0
2,2,-d1.-r0.f0.-r1.f1.d1.-r1.-f0.d1.f0.d1.d1
3,3,-f0.d0.-r0.f0.-d0.-r0.d0.-f0.-r0.-f0
4,4,-r1.-f0.d0.r0.-d1.-d1.r1.d1.f0.r1.-d1.-r1
...,...,...
393,393,f19.f21.-f39.f20.f2.-f5.f7.-r3.f55.-f12.f65.-f...
394,394,-f31.-f22.f16.-f17.-f13.-f24.-f14.f2.f21.f44.f...
395,395,-r0.-f42.-f8.f16.-f49.f14.-f1.f56.f26.f35.f62....
396,396,f25.-f29.f46.f49.-f8.f27.f26.-f20.f2.-f20.f6.f...


In [126]:
# Define the file path for the output CSV file
output_csv_path = 'submission.csv'

# Save the output DataFrame to a CSV file
res_df.to_csv(output_csv_path, index=False)

# Return the path of the saved file
output_csv_path

'submission.csv'